## Dependencies

In [15]:
!pip install pydub SpeechRecognition word2number
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [16]:
from google.colab import output
import re
from word2number import w2n
import nltk
from nltk import word_tokenize, pos_tag, sent_tokenize
import json
from google.colab import files
from IPython.display import display, Javascript
import base64
import speech_recognition as sr
from pydub import AudioSegment
import io

In [17]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [18]:
# Define a global variable to store the transcript
transcript_text = ""

In [19]:
# Function to handle uploaded audio
def upload_audio(audio_data):
    global transcript_text
    file_path = decode_audio(audio_data)
    transcript_text = transcribe_audio(file_path)
    transcript_text = punctuate_text(transcript_text)
    print(f"Transcription: {transcript_text}")

output.register_callback('notebook.upload_audio', upload_audio)

## Record Audio Prompt

Variables needed in Spoken Prompt: **Height, Corner Fillet, Leg Insert, Sides, Thickness**

When speaking your Prompt, remember to make sure that your Variable in the prompt and its associated value are closer together than any other values listed. If they are too close, they will get confused and mixed, i.e, the more detail and words used, the better!

Example: " *Please create a table that has 4 sides, with a height of 3 feet and use a leg insert of 4 inches. Make it 2 inches thick with corner fillets of 4 inches.* "

In [ ]:
# JavaScript code for recording audio
record_audio_js = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))

const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})

let recorder, stream, chunks = []

async function startRecording() {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  start_button.disabled = true
  stop_button.disabled = false
}

function stopRecording() {
  return new Promise(async resolve => {
    recorder.onstop = async () => {
      const blob = new Blob(chunks)
      const audio = await b2text(blob)
      resolve(audio)
    }
    recorder.stop()
    stream.getTracks().forEach(track => track.stop())
    start_button.disabled = false
    stop_button.disabled = true
  })
}

var start_button = document.createElement('button');
start_button.textContent = "Start Recording";
start_button.style.display = "block";
start_button.style.margin = "10px 0";
start_button.onclick = startRecording;

var stop_button = document.createElement('button');
stop_button.textContent = "Stop Recording";
stop_button.style.display = "block";
stop_button.style.margin = "10px 0";
stop_button.disabled = true;
stop_button.onclick = async () => {
  const audio = await stopRecording();
  google.colab.kernel.invokeFunction('notebook.upload_audio', [audio], {});
};

document.body.appendChild(start_button);
document.body.appendChild(stop_button);
"""

display(Javascript(record_audio_js))

def decode_audio(audio_data):
    audio_data = audio_data.split(',')[1]
    audio_bytes = base64.b64decode(audio_data)
    audio = AudioSegment.from_file(io.BytesIO(audio_bytes), format="webm")
    audio.export("output.wav", format="wav")
    return "output.wav"

# Function to transcribe audio
def transcribe_audio(file_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(file_path) as source:
        audio = recognizer.record(source)
    try:
        return recognizer.recognize_google(audio)
    except sr.UnknownValueError:
        return "Could not understand audio"
    except sr.RequestError as e:
        return f"Could not request results; {e}"

# Function to punctuate text
def punctuate_text(text):
    # Split text into sentences
    sentences = sent_tokenize(text)
    punctuated_sentences = []

    for sentence in sentences:
        # Capitalize the first letter of the sentence
        sentence = sentence.capitalize()

        # Convert numbers from words to digits
        sentence = re.sub(r'\b(one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty)\b',
                          lambda match: str(w2n.word_to_num(match.group())), sentence)

        # Add commas before conjunctions if they follow a clause
        sentence = re.sub(r'(\b\w+\b)\s(and|but|or|nor|for|so|yet)\s(\b\w+\b)',
                          r'\1, \2 \3', sentence)

        # Add commas in lists (basic approach for commas in lists)
        sentence = re.sub(r'(\b\w+\b)\s(\b\w+\b)\s(and|or)\s(\b\w+\b)',
                          r'\1, \2, \3 \4', sentence)

        # Remove commas that follow numbers
        sentence = re.sub(r'(\d+),(\s)', r'\1\2', sentence)  # Remove comma directly after numbers

        # Ensure the sentence ends with a period
        if not sentence.endswith('.'):
            sentence += '.'

        punctuated_sentences.append(sentence)

    return ' '.join(punctuated_sentences)

print(transcript_text)


<IPython.core.display.Javascript object>


Transcription: I need to create a table that has a height of 2.75 ft with a thickness of 75 in with 4 sides, and a radius of 3 ft the table must have a corner fillet of 0.2 in with overall edge fillets of 1 inch, for the legs make the leg radius 2 inches with a skew of 5 degrees, and a taper of 3 inches.


In [ ]:
prompt = transcript_text
print(prompt)

I need to create a table that has a height of 2.75 ft with a thickness of 75 in with 4 sides, and a radius of 3 ft the table must have a corner fillet of 0.2 in with overall edge fillets of 1 inch, for the legs make the leg radius 2 inches with a skew of 5 degrees, and a taper of 3 inches.


## Capitalize

In [90]:
#Option to Type Prompt Instead
prompt = "I need to create a table with 3 foot sides and a height of 3 feet and use a leg insert of 4 inches make it 2 inches thick with corner fillets of 4 inches."

In [44]:
# List of specific words to capitalize
words_to_capitalize = ["tall", "height", "thickness", "thick", "radius", "corner fillet", "edge fillets", "leg insert", "high", "sides", "ft", "feet", "inch", "inches", "in", "fillet", "fillets",]

# Tokenize the prompt into words
tokens = word_tokenize(prompt)

# Perform part-of-speech tagging
tagged_tokens = pos_tag(tokens)

# Initialize a list to store capitalized words
capitalized_words = []

In [45]:
# Iterate through tagged tokens
for token, tag in tagged_tokens:
    if (token.lower() in words_to_capitalize or
        tag.startswith('NN')):  # NN for nouns, VB for verbs, JJ for adjectives
        capitalized_words.append(token.capitalize())
    else:
        capitalized_words.append(token)

# Join the words back into a single string
capitalized_prompt = ' '.join(capitalized_words)

# Remove any space before the period at the end
if capitalized_prompt.endswith('.'):
    capitalized_prompt = capitalized_prompt[:-2] + '.'

In [46]:
print("Original Prompt:")
print(prompt)
print("\nCapitalized Prompt:")
print(capitalized_prompt)

Original Prompt:
I need to create a table with 3 foot sides and a height of 3 feet and use a leg insert of 4 inches make it 2 inches thick with corner fillets of 4 inches.

Capitalized Prompt:
I need to create a Table with 3 Foot Sides and a Height of 3 Feet and use a Leg Insert of 4 Inches make it 2 Inches Thick with corner Fillets of 4 Inches.


## Find / Assign Variables for Family Member Creation

In [47]:
# Define possible synonyms for each parameter
parameter_synonyms = {
    "Height": ["height", "tall", "elevation", "high"],
    "Corner Fillet": ["corner fillet", "fillet", "rounded corner", "corner fillets", "fillets"],
    "Leg Insert": ["leg insert", "leg distance", "leg spacing", "inserts", "leg insert distance of "],
    "Sides": ["sides", "edges"],
    "Thickness": ["thickness", "thick", "depth"]
}

In [48]:
# Function to find the closest number to a keyword, considering proximity and grammar
def find_closest_value(keyword, text):
    pattern = re.compile(r'(\b\d+(\.\d+)?\b)\s*(feet|foot|inches|inch|ft|in|inches.|sides)?')
    matches = pattern.finditer(text)
    keyword_position = text.find(keyword)

    closest_distance = float('inf')
    closest_value = None

    for match in matches:
        number, _, unit = match.groups()
        start, end = match.span()

        # Calculate the distance between the keyword and the number
        distance = abs(keyword_position - start)

        # Determine the direction (before or after) of the keyword relative to the number
        if keyword_position < start:  # Keyword before number
            direction = "after"
        else:  # Keyword after number
            direction = "before"

        # Only consider this match if it's closer than any previously found
        if distance < closest_distance:
            closest_distance = distance
            closest_value = f"{number} {unit}".strip()

            # Ensure grammatical correctness by checking direction
            if direction == "after" and closest_distance > len(keyword):
                continue

    return closest_value

# Extract parameters
parameters = {}

for parameter, synonyms in parameter_synonyms.items():
    for synonym in synonyms:
        if synonym in prompt:
            value = find_closest_value(synonym, prompt)
            if value:
                parameters[parameter] = value
                break

# Output the extracted parameters
print(json.dumps(parameters, indent=2))

{
  "Height": "3 feet",
  "Corner Fillet": "4 inches",
  "Leg Insert": "4 inches",
  "Sides": "3 foot",
  "Thickness": "2 inches"
}


## Export JSON for Revit API

In [49]:
parameters

{'Height': '3 feet',
 'Corner Fillet': '4 inches',
 'Leg Insert': '4 inches',
 'Sides': '3 foot',
 'Thickness': '2 inches'}

In [50]:
# Define the filename for the JSON file
json_filename = "family_parameters.json"

In [51]:
# Write the dictionary to a JSON file
with open(json_filename, 'w') as json_file:
    json.dump(parameters, json_file, indent=4)

In [52]:
# Trigger a download of the JSON file in the browser
files.download(json_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Export Dictionary

In [87]:
def convert_to_revit_units(value):
    """Convert values to Revit-compatible units."""
    if 'foot' in value or 'feet' in value:
        number = re.match(r'(\d+)', value).group()
        return f"{number}ft 0in"
    elif 'inch' in value or 'inches' in value:
        number = re.match(r'(\d+)', value).group()
        return f"{number}in "
    return value

def prepare_revit_parameters(parameters):
    # Define the parameter dictionary with mappings to Revit family parameters
    revit_parameters = {
        "Height": {"ParameterName": "Height", "Value": convert_to_revit_units(parameters.get("Height", "")), "Unit": "feet"},
        "Thickness": {"ParameterName": "Thickness", "Value": convert_to_revit_units(parameters.get("Thickness", "")), "Unit": "in"},
        "Sides": {"ParameterName": "Number of Sides", "Value": parameters.get("Sides", ""), "Unit": "in"},
        "Leg Insert": {"ParameterName": "Leg Insert", "Value": convert_to_revit_units(parameters.get("Leg Insert", "")), "Unit": "in"},
        "Corner Fillet": {"ParameterName": "Corner Fillet", "Value": convert_to_revit_units(parameters.get("Corner Fillet", "")), "Unit": "in"}
    }

    return revit_parameters

revit_parameters = prepare_revit_parameters(parameters)

In [64]:
# Define the filename for the JSON file
json_filename = "family_parameters.json"

In [65]:
# Write the dictionary to a JSON file
with open(json_filename, 'w') as json_file:
    json.dump(revit_parameters, json_file, indent=4)

In [89]:
# Output the dictionary to check the structure
print(json.dumps(revit_parameters, indent=4))

{
    "Height": {
        "ParameterName": "Height",
        "Value": "3ft 0in",
        "Unit": "feet"
    },
    "Thickness": {
        "ParameterName": "Thickness",
        "Value": "2in ",
        "Unit": "in"
    },
    "Sides": {
        "ParameterName": "Number of Sides",
        "Value": "3 foot",
        "Unit": "in"
    },
    "Leg Insert": {
        "ParameterName": "Leg Insert",
        "Value": "4in ",
        "Unit": "in"
    },
    "Corner Fillet": {
        "ParameterName": "Corner Fillet",
        "Value": "4in ",
        "Unit": "in"
    }
}


In [57]:
# Trigger a download of the JSON file in the browser
files.download(json_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>